In [2]:
import pandas as pd
df = pd.read_csv("Finland.csv")

In [3]:
df = df.iloc[:, 3:]
df

,Datetime (Local),Price (EUR/MWhe)
0,2015-01-01 02:00:00,23.37
1,2015-01-01 03:00:00,19.33
2,2015-01-01 04:00:00,17.66
3,2015-01-01 05:00:00,17.53
4,2015-01-01 06:00:00,18.07
...,...,...
89889,2025-04-03 19:00:00,-2.12
89890,2025-04-03 20:00:00,-2.02
89891,2025-04-03 21:00:00,-2.03
89892,2025-04-03 22:00:00,-2.42


In [4]:
df.set_index("Datetime (Local)", inplace=True)
df.head(5)

,Price (EUR/MWhe)
Datetime (Local),
2015-01-01 02:00:00,23.37
2015-01-01 03:00:00,19.33
2015-01-01 04:00:00,17.66
2015-01-01 05:00:00,17.53
2015-01-01 06:00:00,18.07


In [5]:
df = df.rename(columns={
    df.columns[0]: "Price"        # renomme dynamiquement la 2e colonne
})

df

,Price
Datetime (Local),
2015-01-01 02:00:00,23.37
2015-01-01 03:00:00,19.33
2015-01-01 04:00:00,17.66
2015-01-01 05:00:00,17.53
2015-01-01 06:00:00,18.07
...,...
2025-04-03 19:00:00,-2.12
2025-04-03 20:00:00,-2.02
2025-04-03 21:00:00,-2.03


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# === Paramètres ===
sequence_length = 168  # 1 semaine d'historique (168 heures)
horizon = 168          # Prédire 1 semaine (168 heures) dans le futur

df.index = pd.to_datetime(df.index)

# Ajouter des features temporelles
df['hour'] = df.index.hour
df['day_of_week'] = df.index.dayofweek
df['is_weekend'] = (df['day_of_week'] >= 5).astype(float)
df['month'] = df.index.month

# Normaliser Price et les features temporelles (sauf is_weekend qui est déjà 0/1)
scaler_price = MinMaxScaler()
df['Price_norm'] = scaler_price.fit_transform(df[['Price']])

scaler_hour = MinMaxScaler()
df['hour_norm'] = scaler_hour.fit_transform(df[['hour']])

scaler_day = MinMaxScaler()
df['day_norm'] = scaler_day.fit_transform(df[['day_of_week']])

scaler_month = MinMaxScaler()
df['month_norm'] = scaler_month.fit_transform(df[['month']])

# Définir les colonnes de features à utiliser pour le modèle
feature_cols = ['Price_norm', 'hour_norm', 'day_norm', 'is_weekend', 'month_norm']

# === 2. Créer les séquences pour le LSTM ===
X, y = [], []
for i in range(sequence_length, len(df) - horizon):
    # Séquence d'entrée : on prend 'sequence_length' heures d'historique pour toutes les features
    X.append(df.iloc[i - sequence_length:i][feature_cols].values)
    # La cible est les 'horizon' heures suivantes, ici on prédit uniquement le Price (normalisé)
    y.append(df.iloc[i:i + horizon]['Price_norm'].values)

X = np.array(X)  # forme attendue : (n_samples, 168, n_features)
y = np.array(y)  # forme attendue : (n_samples, 168)

print("Forme de X :", X.shape)
print("Forme de y :", y.shape)

# === 3. Découper en ensembles d'entraînement et de test (en respectant l'ordre chronologique) ===
split_idx = int(len(X) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

# === 4. Construire un modèle LSTM avancé ===
model = Sequential()
# Couche 1 : Bidirectional LSTM avec retour des séquences
model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=(sequence_length, len(feature_cols))))
model.add(Dropout(0.2))
# Couche 2 : Seconde couche Bidirectional LSTM sans retour de séquence
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.2))
# Couche Dense intermédiaire
model.add(Dense(128, activation='relu'))
# Couche de sortie : prédiction multi-sortie (168 valeurs)
model.add(Dense(horizon))
model.compile(optimizer='adam', loss='mse')
model.summary()

# === 5. Entraîner le modèle avec EarlyStopping ===
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(
    X_train, y_train,
    epochs=200,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[early_stop],
    verbose=1
)

# === 6. Prédiction sur le jeu de test ===
y_pred = model.predict(X_test)

# === 7. Inverser la normalisation pour obtenir des valeurs réelles ===
# Note : On inverse uniquement pour la target (Price)
y_pred_inv = scaler_price.inverse_transform(y_pred)
y_test_inv = scaler_price.inverse_transform(y_test)

# === 8. Calculer les métriques (globales sur l'ensemble du test) ===
# On a des sorties multi-step, donc on a un tableau 2D ; on les aplatis pour un calcul global.
r2 = r2_score(y_test_inv.flatten(), y_pred_inv.flatten())
mae = mean_absolute_error(y_test_inv.flatten(), y_pred_inv.flatten())
rmse = np.sqrt(mean_squared_error(y_test_inv.flatten(), y_pred_inv.flatten()))

print("R² :", r2)
print("MAE :", mae)
print("RMSE :", rmse)

# === 9. Visualiser la prédiction pour le premier échantillon du test ===
plt.figure(figsize=(12, 6))
plt.plot(y_test_inv[0], label='Prix Réel')
plt.plot(y_pred_inv[0], label='Prix Prévu')
plt.title("Prédiction sur 1 semaine (168h) avec features additionnelles\n(1er échantillon test)")
plt.xlabel("Heures dans le futur")
plt.ylabel("Prix (€/MWh)")
plt.legend()
plt.tight_layout()
plt.show()


Forme de X : (89558, 168, 5)
Forme de y : (89558, 168)


2025-04-05 07:05:39.529363: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-04-05 07:05:39.529542: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-04-05 07:05:39.529546: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-04-05 07:05:39.529708: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-05 07:05:39.529719: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/ethanbareille/miniconda3/envs/ml_env/lib/python3.11/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a lay

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)   │ (None, 168, 256)       │       137,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 168, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 128)            │       164,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 168)            │        21,672 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 339,752 (1.30 MB)

...

 Non-trainable params: 0 (0.00 B)

Epoch 1/200


2025-04-05 07:05:40.631294: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


 217/2239 ━━━━━━━━━━━━━━━━━━━━ 3:12 95ms/step - loss: 0.0093

KeyboardInterrupt: 